In [ ]:
import subprocess

def download_youtube_video(url):
    command = ["yt-dlp", "-o", "%(title)s.%(ext)s", url]
    subprocess.run(command)

download_youtube_video("https://www.youtube.com/watch?v=3wk45Ow3m3M")


In [4]:
import subprocess
import os
import glob

save_path = "./.cache/"

def download_youtube_video(url, save_path):
    os.makedirs(save_path, exist_ok=True)
    output_template = os.path.join(save_path, "%(title)s.%(ext)s")
    subprocess.run(["yt-dlp", "-o", output_template, url])
    
    # 다운로드된 실제 파일 찾기
    downloaded_files = glob.glob(os.path.join(save_path, "*"))
    if not downloaded_files:
        raise FileNotFoundError("다운로드된 파일을 찾을 수 없습니다.")
    
    # 최신 파일 반환 (다운로드된 파일이 여러 개일 경우)
    downloaded_files.sort(key=os.path.getmtime, reverse=True)
    return downloaded_files[0]

download_youtube_video("https://www.youtube.com/watch?v=3wk45Ow3m3M", save_path)

'./.cache\\파이썬으로 AI Agent 만들기 (프레임워크 X).webm'

In [2]:
def extract_audio(video_path, audio_path):
    command = ["ffmpeg", "-i", video_path, "-vn", audio_path]
    subprocess.run(command)

extract_audio("./files/video.webm", "./files/audio.mp3")

In [ ]:
from pydub import AudioSegment

track = AudioSegment.from_mp3("./files/audio.mp3")

2583.7013333333334

In [ ]:
import math
from pydub import AudioSegment

def cut_audio_in_chunks(audio_path, unit_minute, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = unit_minute * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)
    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len

        chunk = track[start_time:end_time]

        chunk.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")

cut_audio_in_chunks("./files/audio.mp3", 10, "./files/chunks")

In [13]:
from typing import final
import openai
import glob


def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3") # 해당 폴더 안에서 패턴과 맞는 모든 파일 경로를 리스트로 반환
    final_transcript = ""
    for file in files:
        with open(file, "rb") as audio_file:
            transcript = openai.Audio.transcribe("whisper-1", audio_file)
            final_transcript += transcript["text"]
    with open(destination, "w", encoding="utf-8") as file:
        file.write(final_transcript)

transcribe_chunks("./files/chunks", "./files/transcript.txt" )